In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../Data/train.csv")


In [7]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


df = pd.read_csv("../Data/train.csv")


target = "Depression_Score"
X = df.drop(columns=[target])
y = df[target]


numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

print("Numeric columns:", list(numeric_features))
print("Categorical columns:", list(categorical_features))

# Define preprocessing for each type
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


ridge = Ridge(random_state=42)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", ridge),
])

# Hyperparameter grid for Ridge
param_grid = {
    "model__alpha": [0.1, 1.0, 10.0, 100.0]
}

# GridSearchCV with cross-validation
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid.fit(X, y)

print("Best params:", grid.best_params_)
print("Best CV (neg MSE):", grid.best_score_)
print("Best CV RMSE:", np.sqrt(-grid.best_score_))


Numeric columns: ['Age', 'Sleep_Hours', 'Physical_Activity_Hrs', 'Social_Support_Score', 'Anxiety_Score', 'Stress_Level', 'Family_History_Mental_Illness', 'Chronic_Illnesses', 'Therapy', 'Meditation', 'Financial_Stress', 'Work_Stress', 'Self_Esteem_Score', 'Life_Satisfaction_Score', 'Loneliness_Score']
Categorical columns: ['Gender', 'Education_Level', 'Employment_Status', 'Medication_Use', 'Substance_Use']
Best params: {'model__alpha': 100.0}
Best CV (neg MSE): -33.17186065893677
Best CV RMSE: 5.759501771762621
